In [16]:
import numpy as np
import pandas as ps

#4-стадийный метод РК четвертого порядка точности
def f1(x,y):
    return  y/x - y*y #по сути y'
def f2(x,y,h):
    return f1(x + h/2, y + f1(x,y)*h/2)
def f3(x,y,h):
    return f1(x + h/2, y + f2(x,y,h)*h/2)
def f4(x,y,h):
    return f1(x + h, y + f3(x,y,h)*h)
    
#функция, выводящая n равноудаленных значений у на (a,b); q - параметр, для уменьшения шага в q раз
def RK4(x0,y0,a,b,n,q):
    h = (b - a)/(q*n) #шаг
    array = np.zeros((2,q*n+1)) #1 строка - х, 2 - у - сеточная функция с шагом h/q
    array[0][0] = x0 
    array[1][0] = y0
    for i in range(1,q*n+1):
        array[0][i] = array[0][i-1] + h
        array[1][i] = array[1][i-1] + h*(f1(array[0][i-1],array[1][i-1]) + 2*f2(array[0][i-1],array[1][i-1],h) + 2*f3(array[0][i-1],array[1][i-1],h) + f4(array[0][i-1],array[1][i-1],h))/6
    
    #таблица для вывода(ведь нужно вывести не всю сеточную функцию, а n равноудаленных значений)
    xy = np.zeros((2,n+1))
    for i in range(0,n+1):
        xy[0,i] = array[0,q*i]
        xy[1,i] = array[1,q*i]
    return xy



def function(x,y,a,b,n):
    mas = np.zeros((8,n+1))
    for i in range(0,n+1):
        mas[0][i] = RK4(x,y,a,b,n,1)[1][i]
        mas[1][i] = RK4(x,y,a,b,n,2)[1][i]
        mas[2][i] = RK4(x,y,a,b,n,4)[1][i]
        mas[3][i] = RK4(x,y,a,b,n,8)[1][i]
        mas[4][i] = RK4(x,y,a,b,n,16)[1][i]
        mas[5][i] = RK4(x,y,a,b,n,32)[1][i]
        mas[6][i] = RK4(x,y,a,b,n,64)[1][i]
        mas[7][i] = RK4(x,y,a,b,n,1)[0][i]

    table = ps.DataFrame(data=mas, index=np.arange(0,8), columns=np.arange(0,n+1)) 
    table.index = ['l = 11','l = 21','l = 41','l = 81','l = 161','l = 221','l = 441','x']
    return mas

In [41]:
def epsilon(x,y,a,b,n,j):
    s = np.zeros(n)
    #вычитаю из аналитического решения решение по алгоритму для каждого шага
    for i in range(n):
        s[i] = np.fabs(RK4(x,y,a,b,n,32)[1][i] - RK4(x,y,a,b,n,2**j)[1][i]) 
    return s

a = []
for j in range(5):
    a.append(max(epsilon(1,-0.25,1,3,11,j)))
aa = [11,21,41,81,161]
aaa = np.zeros((2,5))
aaa[0][:]=aa
aaa[1][:]=a
table = ps.DataFrame(data=aaa, index=np.arange(0,2), columns=np.arange(0,5))
table.columns = [' ',' ',' ',' ',' ']
table.index = ['число узлов','последовательность']
table

,,,,,
число узлов,11.000000,21.000000,41.000000,81.000000,161.000000
последовательность,0.040267,0.003859,0.000285,0.000019,0.000001


In [9]:
function(1,-0.25,1,3,11)

,0,1,2,3,4,5,6,7,8,9,10,11
l = 11,-0.25,-0.310868,-0.381942,-0.467495,-0.574166,-0.712943,-0.903529,-1.185239,-1.649641,-2.570473,-5.287858,-44.865339
l = 21,-0.25,-0.310869,-0.381944,-0.467500,-0.574175,-0.712962,-0.903568,-1.185337,-1.649975,-2.572423,-5.324266,-89.993193
l = 41,-0.25,-0.310870,-0.381944,-0.467500,-0.574176,-0.712963,-0.903571,-1.185344,-1.649998,-2.572570,-5.327840,-180.199298
l = 81,-0.25,-0.310870,-0.381944,-0.467500,-0.574176,-0.712963,-0.903571,-1.185345,-1.650000,-2.572580,-5.328106,-360.587012
l = 161,-0.25,-0.310870,-0.381944,-0.467500,-0.574176,-0.712963,-0.903571,-1.185345,-1.650000,-2.572581,-5.328124,-721.351003
l = 221,-0.25,-0.310870,-0.381944,-0.467500,-0.574176,-0.712963,-0.903571,-1.185345,-1.650000,-2.572581,-5.328125,-1442.873640
l = 441,-0.25,-0.310870,-0.381944,-0.467500,-0.574176,-0.712963,-0.903571,-1.185345,-1.650000,-2.572581,-5.328125,-2885.916372
x,1.00,1.181818,1.363636,1.545455,1.727273,1.909091,2.090909,2.272727,2.454545,2.636364,2.818182,3.000000


In [17]:
#ф-я вычитает из аналитического решения j строчку mas
def epsilon(n,j):
    s = np.zeros(n)
    #вычитаю из аналитического решения решение по алгоритму для каждого шага
    for i in range(n):
        s[i] = np.fabs(function(1,-0.25,1,3,11)[5][i] - function(1,-0.25,1,3,11)[j][i]) 
    return s

epsilon(11,1)

array([0.00000000e+00, 7.74388796e-08, 1.94193671e-07, 3.84583417e-07,
       7.23212668e-07, 1.39509095e-06, 2.94327173e-06, 7.38175247e-06,
       2.53066356e-05, 1.57276312e-04, 3.85894148e-03])

In [37]:
b = np.zeros((2,11))
b[1][:]= RK4(1,-0.25,1,3,11,32)[1][:11]
b[0][:]=RK4(1,-0.25,1,3,11,32)[0][:11]
table = ps.DataFrame(data=b, index=np.arange(0,2), columns=np.arange(0,11))
#table.columns = [' ',' ',' ',' ',' ']
table.index = ['x','y']
table

,0,1,2,3,4,5,6,7,8,9,10
x,1.00,1.181818,1.363636,1.545455,1.727273,1.909091,2.090909,2.272727,2.454545,2.636364,2.818182
y,-0.25,-0.310870,-0.381944,-0.467500,-0.574176,-0.712963,-0.903571,-1.185345,-1.650000,-2.572581,-5.328125


In [39]:
np.fabs(-3)

3.0